In [ ]:
# 라이브러리 설치 
# !pip install torch torchvision torchaudio

In [ ]:
# 라이브러리 로드 
import torch 
import torch.nn as nn
import torch.optim as optim

In [ ]:
# 데이터셋을 생성 
# 독립, 종속 -> 파이토치에서는 독립, 종속 변수의 데이터의 타입은 Tensor 로 구성
# 독립 변수 -> 2차원 데이터
# 종속 변수 -> 2차원 데이터 (sklearn에서는 1차원)
X = torch.tensor( [ [1.0], [2.0], [3.0], [4.0] ] )
# 종속 변수는 독립 변수 데이터에서 2를 곱하고 1을 더해준 값으로 생성
Y = torch.tensor( [ [3.0], [5.0], [7.0], [9.0] ] )

In [ ]:
type(X)

In [ ]:
# 선형 회귀 
class LinearReg(nn.Module):
    # 생성자 함수 
    def __init__(self):
        # self : 클래스가 생성된 메모리의 주소
        # super() : 부모 클래스(nn.Module)를 의미 -> 상속
        super( LinearReg, self ).__init__()     # 부모 클래스의 생성자 함수를 실행
        # Linear()에서는 첫번째인자로는 입력 데이터(독립의 피쳐) 크기 
        # 두번째 인자의 출력의(종속의 피쳐) 크기
        self.linear = nn.Linear(1, 1)

    def forward(self, x):
        return self.linear(x)

In [ ]:
# 모델을 생성 -> 회귀 모델 
# LinearReg 라는 클래스에서 생성자 함수에 매개변수가 self을 제외하고 존재 하지 않기 때문에 
# 클래스 생성 시 인자 값을 넣지 않는다. 
model = LinearReg()

In [ ]:
# 손실 함수 
criterion = nn.MSELoss()
# 가중치의 갱신 (경사 하강법) lr은 가중치의 변환 시 변환 비율
optimizer = optim.SGD(model.parameters(), lr=0.01)

In [ ]:
# 500번 반복 실행하면서 Loss를 확인 
for epoch in range(500):
    # 순전파 
    Y_pred = model(X)
    # 손실 함수
    loss = criterion(Y_pred, Y)
    # 역전파
    optimizer.zero_grad()       # 기울기 초기화
    loss.backward()             # 기울기 계산
    optimizer.step()            # 파타미터를 업데이트

    # 50회 마다 loss의 값을 출력 
    if (epoch + 1) % 50 == 0:
        print(f'Epoch : [{epoch+1}, 500], Loss : {round(loss.item(), 4)}')

In [ ]:
# 예측 값 확인 
pred = model(X).detach()

for i in range(len(X)):
    print(f"독립 : {X[i].item()}, 종속 : {Y[i].item()}, 예측 : {pred[i].item()}")

In [ ]:
# sklearn에서 제공하는 캘리포니아 데이터를 이용하여 회귀분석 
from sklearn.datasets import fetch_california_housing
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import numpy as np

In [134]:
# 데이터를 로드해서 독립 종속 변수 데이터 생성 
data = fetch_california_housing()

X = data['data']
Y = data['target']

print("독립 변수 데이터의 크기 : ", X.shape)
print('종속 변수 데이터의 크기 : ', Y.shape)

독립 변수 데이터의 크기 :  (20640, 8)
종속 변수 데이터의 크기 :  (20640,)


In [135]:
# 데이터를 학습, 평가 데이터로 분할
X_train, X_test, Y_train, Y_test = train_test_split(
    X, Y, test_size=0.2, random_state=42
)

In [136]:
# StandardScaler를 이용하여 스케일링 -> train 데이터를 기준으로 fitting을 하고 
# train, test 모두 같은 fit model에서 변환 작업 (데이터 누수 방지)
scaler = StandardScaler()
X_train_sc = scaler.fit_transform(X_train)
X_test_sc = scaler.transform(X_test)

In [137]:
# 총 4개의 데이터의 타입은 array형태 -> Tensor 형태로 변환 
X_train_sc_tensor = torch.tensor(X_train_sc, dtype=torch.float32)
X_test_sc_tensor = torch.tensor(X_test_sc, dtype=torch.float32)
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
# Y의 데이터들은 현재 1차원-> 2차원 변경해서  Tensor로 변환
Y_train_tensor = torch.tensor( Y_train.reshape(-1, 1), dtype=torch.float32 )
Y_test_tensor = torch.tensor(Y_test.reshape(-1, 1), dtype=torch.float32)

In [138]:
# class 생성 
class Reg(nn.Module):
    # 생성자 함수에서 self외의 매개변수를 하나 생성 -> 
    # nn.Linear()에서 사용할 첫번째 인자 값 (입력 값의 열의 크기 )
    def __init__(self, _dim):
        super(Reg, self).__init__()
        self.linear = nn.Linear(_dim, 1)
    
    def forward(self, x):
        return self.linear(x)

In [139]:
# 모델을 생성 -> 학습 데이터의 피쳐의 개수
reg_model = Reg( X_train.shape[1] )

In [140]:
# 손실 함수 
criterion = nn.MSELoss()
# 가중치 변환
optimizer = optim.SGD(reg_model.parameters(), lr= 0.01)

In [141]:
# 학습 반복 실행 
for epoch in range(300):
    # 순전파
    Y_pred = reg_model(X_train_sc_tensor)
    # 손실
    loss = criterion(Y_pred, Y_train_tensor)
    # 기울기 초기화 
    optimizer.zero_grad()
    # 역전파 
    loss.backward()
    # 가중치 업데이트 
    optimizer.step()
    # 30회마다 loss의 변화 확인
    if (epoch + 1) % 30 == 0:
        print(f'Epoch : {epoch+1}, Loss : {round(loss.item(), 4)}')

Epoch : 30, Loss : 2.224
Epoch : 60, Loss : 1.1339
Epoch : 90, Loss : 0.7977
Epoch : 120, Loss : 0.6875
Epoch : 150, Loss : 0.6461
Epoch : 180, Loss : 0.6265
Epoch : 210, Loss : 0.6143
Epoch : 240, Loss : 0.6049
Epoch : 270, Loss : 0.5969
Epoch : 300, Loss : 0.5899


In [142]:
# 현재 모델의 학습 모드에서 평가 모드로 변환 (학습이 중지가 아니라 모드의 변환)
# eval() : 평가모드 변환
# train() : 학습모드 변환
reg_model.eval()

Reg(
  (linear): Linear(in_features=8, out_features=1, bias=True)
)

In [143]:
# 예측과 평가 
# torch.nograd() -> 평가시 gradient 계산을 비활성화 (속도 빨라짐, 메모리 소모 줄어듬)
with torch.no_grad():
    Y_pred = reg_model(X_test_sc_tensor)
    test_loss = criterion(Y_pred, Y_test_tensor)

In [144]:
# 성능 평가 
# MSE, RMSE 지표 확인 
rmse = np.sqrt(test_loss.item())
print("테스트 데이터의 MSE : ", round(test_loss.item(), 4))
print("테스트 데이터의 RSME : ", round(rmse, 4))

테스트 데이터의 MSE :  0.6035
테스트 데이터의 RSME :  0.7768


In [145]:
# 실제 값과 예측 값을 확인 -> 상위 10개의 데이터만 확인
for i in range(10):
    print(f"실제 값 : {Y_test[i]}, 예측 값 : {round(Y_pred[i].item(), 3)}")

실제 값 : 0.477, 예측 값 : 0.972
실제 값 : 0.458, 예측 값 : 1.586
실제 값 : 5.00001, 예측 값 : 2.378
실제 값 : 2.186, 예측 값 : 2.72
실제 값 : 2.78, 예측 값 : 2.205
실제 값 : 1.587, 예측 값 : 2.139
실제 값 : 1.982, 예측 값 : 2.709
실제 값 : 1.575, 예측 값 : 2.177
실제 값 : 3.4, 예측 값 : 2.152
실제 값 : 4.466, 예측 값 : 4.12


In [146]:
# 스탠다드 스케일링을 하지 않은 데이터를 이용하여 학습하고 예측을 출력하시오


# 모델 생성 
reg_model2 = Reg(X_train.shape[1])
print(reg_model2)

Reg(
  (linear): Linear(in_features=8, out_features=1, bias=True)
)


In [147]:
criterion2 = nn.MSELoss()
# Adam -> 적응적 학습율 기반 경사하강법 
optimizer2 = optim.Adam(reg_model2.parameters(), lr = 0.01)

In [148]:
for epoch in range(300):
    Y_pred2 = reg_model2(X_train_tensor)
    # print(Y_pred2)
    # break
    loss2 = criterion2(Y_pred2, Y_train_tensor)

    optimizer2.zero_grad()

    loss2.backward()

    optimizer2.step()

    if (epoch + 1) % 30 == 0:
        print(f'Epoch : {epoch+1}, Loss : {round(loss2.item(), 4)}')

Epoch : 30, Loss : 1734.6365
Epoch : 60, Loss : 83.7378
Epoch : 90, Loss : 31.3308
Epoch : 120, Loss : 9.9788
Epoch : 150, Loss : 3.7891
Epoch : 180, Loss : 2.1297
Epoch : 210, Loss : 1.6199
Epoch : 240, Loss : 1.3997
Epoch : 270, Loss : 1.2743
Epoch : 300, Loss : 1.1903


In [149]:
reg_model2.eval()
with torch.no_grad():
    Y_pred2 = reg_model2(X_test_tensor)
    test_loss2 = criterion2(Y_pred2, Y_test_tensor).item()

In [ ]:
# 실제 값과 예측 값 확인 
for i in range(10):
    print(f"실제 : {Y_test[i]}, 예측 : {round(Y_pred2[i].item(), 4)}")

In [151]:
rmse2 = np.sqrt(test_loss2)
print("스케일링 작업x 테스트 MSE : ", round(test_loss2, 4))
print("스케일링 작업x 테스트 RMSE : ", round(rmse2, 4))

스케일링 작업x 테스트 MSE :  1.1597
스케일링 작업x 테스트 RMSE :  1.0769


In [152]:
# 파이토치를 이용한 분류 분석 
# iris 데이터를 이용
from sklearn.datasets import load_iris
# 분류 검증 정확도 
from sklearn.metrics import accuracy_score

In [153]:
iris = load_iris()
X = iris['data']
Y = iris['target']

In [154]:
# train, test 분할 
X_train, X_test, Y_train, Y_test = train_test_split(
    X, Y, test_size=0.2, random_state=42
)

In [155]:
# 스탠다드스케일러를 이용한 스케일링
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [156]:
# 분류 모델에서 Tensor 데이터로 변환 
# 독립변수 -> float32 타입으로 변경
# 종속변수 -> long 타입으로 변경
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
Y_train_tensor = torch.tensor(Y_train, dtype=torch.long)
Y_test_tensor = torch.tensor(Y_test, dtype=torch.long)

In [157]:
Y_train_tensor

tensor([0, 0, 1, 0, 0, 2, 1, 0, 0, 0, 2, 1, 1, 0, 0, 1, 2, 2, 1, 2, 1, 2, 1, 0,
        2, 1, 0, 0, 0, 1, 2, 0, 0, 0, 1, 0, 1, 2, 0, 1, 2, 0, 2, 2, 1, 1, 2, 1,
        0, 1, 2, 0, 0, 1, 1, 0, 2, 0, 0, 1, 1, 2, 1, 2, 2, 1, 0, 0, 2, 2, 0, 0,
        0, 1, 2, 0, 2, 2, 0, 1, 1, 2, 1, 2, 0, 2, 1, 2, 1, 1, 1, 0, 1, 1, 0, 1,
        2, 2, 0, 1, 2, 2, 0, 2, 0, 1, 2, 2, 1, 2, 1, 1, 2, 2, 0, 1, 2, 0, 1, 2])

In [158]:
X.shape

(150, 4)

In [159]:
# 모델의 정의 
class IrisClass(nn.Module):
    def __init__(self):
        # 부모 클래스의 생성자 함수 실행
        super(IrisClass, self).__init__()
        # 분류 모델을 선택 
        self.model = nn.Sequential(
            # Linear() -> 첫번째 인자 입력 값의 열의 개수
            # 두번째 인자 값을 분류 클래스의 개수
            nn.Linear(4, 3)
        )
    
    def forward(self, x):
        return self.model(x)

In [160]:
# 모델의 생성
cls_model = IrisClass()

In [161]:
# 손실 함수 / 옵티마이저 생성 
# 분류 모델의 손실 함수 
criterion = nn.CrossEntropyLoss()
# 옵티마이저 
optimizer = optim.Adam(cls_model.parameters(), lr=0.01)

In [163]:
# 반복 학습 시작
for epoch in range(300):
    Y_pred = cls_model(X_train_tensor)
    loss = criterion(Y_pred, Y_train_tensor)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    # 30회 마다 loss를 확인 
    if (epoch + 1) % 30 == 0:
        print(f"Epoch : {epoch+1}, Loss : {round(loss.item(), 4)}")

Epoch : 30, Loss : 0.5508
Epoch : 60, Loss : 0.4176
Epoch : 90, Loss : 0.3581
Epoch : 120, Loss : 0.3173
Epoch : 150, Loss : 0.2849
Epoch : 180, Loss : 0.258
Epoch : 210, Loss : 0.2355
Epoch : 240, Loss : 0.2164
Epoch : 270, Loss : 0.2002
Epoch : 300, Loss : 0.1863


In [169]:
# 평가 
cls_model.eval()
with torch.no_grad():
    # pred -> 예측에 대한 엔트로피 계수(높은 값의 위치가 유력)
    pred = cls_model(X_test_tensor)
    _, pred_idx = torch.max(pred, 1)
    # 정확도를 계산
    acc = accuracy_score(Y_test, pred_idx)
print("정확도 :", acc)


정확도 : 1.0


In [ ]:
# torch.max() : -> 2차원 리스트 형태의 데이터에서 각 원소별 위치, 값을 되돌려주는 함수
torch.max(pred, 1)

In [170]:
for i in range(10):
    print(f"실제 : {Y_test[i]}, 예측 : {pred_idx[i].item()}")

실제 : 1, 예측 : 1
실제 : 0, 예측 : 0
실제 : 2, 예측 : 2
실제 : 1, 예측 : 1
실제 : 1, 예측 : 1
실제 : 0, 예측 : 0
실제 : 1, 예측 : 1
실제 : 2, 예측 : 2
실제 : 1, 예측 : 1
실제 : 1, 예측 : 1


### 문제
- data 폴더 안에 bodyPerformance.csv 파일을 로드 
- 데이터중 object 형 데이터의 컬럼들은 LabelEncoder를 이용하여 라벨링
- 독립, 종속 변수로 데이터를 나눠준다. 
- train, test 데이터를 8:2의 비율로 나눠준다. 
- 독립 데이터들은 StandardScaler를 이용하여 스케일링
- 생성된 변수 4개를 Tensor의 형태로 변환
- torch의 분류 분석을 통해서 데이터 학습 
    - 손실함수는 Adam을 사용하고 lr의 값은 0.01로 설정 
    - 반복 학습의 횟수는 200회 loss값 확인
- 반복 학습이 끝나면 정확도를 확인
